In [2]:
from networkx import networkx as nx
import matplotlib.pyplot as plt
import scipy
from data_reader import get_stackoverflow_data

BASE_PATH = 'data'

G=nx.Graph()

xml_dict = get_stackoverflow_data(f"{BASE_PATH}/bitcoin")

# example on how to use the xml
for row in xml_dict['Users'][:2]:
    # row objects have tag & attrib
    print(row.tag, row.attrib)
    
# handling the Posts file
posts_dict = dict()
edges = []

for post in xml_dict['Posts']:
    attribs = post.attrib
    user_id = attribs.get('OwnerUserId', attribs.get('OwnerDisplayName'))
    # PostTypeId == 1 indicates that it's a question
    if attribs['PostTypeId'] == '1':
        posts_dict[attribs['Id']] = user_id
    # otherwise it's an answer to an existing question
    elif attribs['PostTypeId'] == '2':
        # fetches the id of the OP and links them
        parent_user_id = posts_dict.get(attribs['ParentId'])
        edges.append((parent_user_id, user_id))
        
        
    
# TODO: read data to graphx

row {'Id': '-1', 'Reputation': '1', 'CreationDate': '2011-08-30T14:52:14.437', 'DisplayName': 'Community', 'LastAccessDate': '2011-08-30T14:52:14.437', 'WebsiteUrl': 'http://meta.stackexchange.com/', 'Location': 'on the server farm', 'AboutMe': '<p>Hi, I\'m not really a person.</p>\n\n<p>I\'m a background process that helps keep this site clean!</p>\n\n<p>I do things like</p>\n\n<ul>\n<li>Randomly poke old unanswered questions every hour so they get some attention</li>\n<li>Own community questions and answers so nobody gets unnecessary reputation from them</li>\n<li>Own downvotes on spam/evil posts that get permanently deleted</li>\n<li>Own suggested edits from anonymous users</li>\n<li><a href="http://meta.stackexchange.com/a/92006">Remove abandoned questions</a></li>\n</ul>\n', 'Views': '0', 'UpVotes': '976', 'DownVotes': '11458', 'AccountId': '-1'}
row {'Id': '2', 'Reputation': '101', 'CreationDate': '2011-08-30T20:53:10.447', 'DisplayName': 'Nick Craver', 'LastAccessDate': '2018-10

In [3]:
edges

[('12', '22'),
 ('12', '9'),
 ('12', '8'),
 ('12', '27'),
 ('12', '16'),
 ('9', '14'),
 ('12', '10'),
 ('12', '9'),
 ('16', '14'),
 ('9', '27'),
 ('12', '8'),
 ('9', '16'),
 ('15', '9'),
 ('21', '27'),
 ('9', '22'),
 ('15', '8'),
 ('6', '27'),
 ('18', '42'),
 ('16', '27'),
 ('5', '22'),
 ('5', '14'),
 ('12', '4774'),
 ('44', '44'),
 ('44', '14'),
 ('6', '9'),
 ('38', '14'),
 ('38', '22'),
 ('21', '54'),
 ('46', '27'),
 ('38', '42'),
 ('16', '56'),
 ('12', '26'),
 ('18', '27'),
 ('38', '7'),
 ('46', '22'),
 ('38', '44'),
 ('18', '9'),
 ('5', '56'),
 ('44', '42'),
 ('21', '22'),
 ('38', '46'),
 ('38', '22'),
 ('58', '9'),
 ('58', '14'),
 ('60', '22'),
 ('24', '14'),
 ('60', '14'),
 ('38', '42'),
 ('5', '53'),
 ('24', '38'),
 ('38', '9'),
 ('44', '44'),
 ('38', '54'),
 ('58', '22'),
 ('60', '27'),
 ('24', '6'),
 ('5', '14'),
 ('5', '27'),
 ('5', '46'),
 ('12', '54'),
 ('44', '22'),
 ('38', '23'),
 ('44', '42'),
 ('46', '7'),
 ('25', '38'),
 ('67', '46'),
 ('38', '22'),
 ('67', '63'),
 ('3